In [1]:
import pandas as pd
import numpy as np
import json
import trimesh
from tqdm import tqdm

In [5]:
symm_gt = {}

# load the two csv files
with open("symm/candidate_planes.json") as json_file:
    candidate_planes = json.load(json_file)
candidate_planes = np.array(candidate_planes)
df_shapenet = pd.read_csv("symm/output_shapenet_pc_axis_v2.csv", header=None)
df_obja_lvis = pd.read_csv("symm/output_obja_lvis_pc_axis_v2.csv", header=None)

# Iterate over the dataframe
datasets = [{"df": df_shapenet, "threshold": 0.02, "subset": "ShapeNet55"}, {"df": df_obja_lvis, "threshold": 0.06, "subset": "objaverse_LVIS"}]
for dataset in datasets:
    df = dataset["df"]
    threshold = dataset["threshold"]
    subset = dataset["subset"]
    for index, row in tqdm(df.iterrows(), total=len(df)):
        filename = row[0]  # Get the filename from the 0th column
        score = row[1:]  # Get the score from the other columns
        
        # Get the indices of the scores that are above the threshold
        indices = np.where(score < threshold)[0].tolist()

        # Get the center and scale of the object
        pc_fname = f"train_data/{subset}/pointclouds/{filename}"
        pc = np.load(pc_fname)
        trimeshpc = trimesh.PointCloud(pc)
        sphere = trimeshpc.bounding_sphere
        radius = (sphere.bounds[1] - sphere.bounds[0]) * 0.5
        radius = radius.tolist()[0]
        center = sphere.center.tolist()

        symm_gt[filename] = {
            "center": center,
            "radius": radius,
            "normal": candidate_planes[indices].tolist()
        }

  0%|          | 75/52896 [00:06<1:16:24, 11.52it/s]


KeyboardInterrupt: 

In [6]:
symm_gt

{'03691459/03691459_8c919e992db7cb081e5b68ee10358ea3.npy': {'center': [-0.00806077103345948,
   -0.0003399450180481822,
   9.725352394956976e-05],
  'radius': [0.6686643520130415, 0.6686643520130415, 0.6686643520130415],
  'normal': [[-1.8369701987210297e-16, -1.0, 6.123233995736766e-17],
   [-1.2246467991473532e-16, 1.4997597826618576e-32, -1.0]]},
 '03691459/03691459_f2e521a8d08e2a2f20363e2561dd589a.npy': {'center': [0.0012024744470690019,
   0.008123691763874619,
   -0.0010565463150899457],
  'radius': [0.6985920406570904, 0.6985920406570904, 0.6985920406570904],
  'normal': [[-1.0, 1.2246467991473532e-16, 6.123233995736766e-17]]},
 '03691459/03691459_b0f209faa41b8f0879da6431b0766445.npy': {'center': [2.2965418542408855e-05,
   0.0075964757168562325,
   4.518495969607628e-06],
  'radius': [0.6308675482992583, 0.6308675482992583, 0.6308675482992583],
  'normal': [[-1.0, 1.2246467991473532e-16, 6.123233995736766e-17],
   [-1.2246467991473532e-16, 1.4997597826618576e-32, -1.0]]},
 '036

In [3]:
list(symm_gt.keys())[-10:]

['03691459/03691459_6e6b4d2603930dfd4fc88114be3a6b6f.npy',
 '03691459/03691459_20fa8d1542e7f835efaf604fa2300241.npy',
 '03691459/03691459_132257fddbebc02ff14235268cca03a3.npy',
 '03691459/03691459_6f1913cf723bdfbec5e5fd334d8e3b16.npy',
 '03691459/03691459_813bb87d0679eeb9e4f8604d0992a17a.npy',
 '03691459/03691459_d6ad58677b397894224f9d60ad32613.npy',
 '03691459/03691459_9a017f96829a34a2b17f090763e2921e.npy',
 '03691459/03691459_5b9b77ad6ce283a398e4bc44d45a32e.npy',
 '03691459/03691459_36b29f93deaf22f66b1cc4d44837f930.npy',
 '03691459/03691459_f0b4735b45d1a6b51aadbcf355bd688f.npy']

In [4]:
json.dump(symm_gt, open("symm/symm_gt.json", "w"))

In [5]:
# import random
# def gen_command(k, v, threshold=0.015):
#     mpath = k
#     planes = v.tolist()
#     command = f"python debug_vis_obja_pc.py {mpath} {str(planes).replace(' ', '')}"
#     # print(command)
#     return command

# commands = []
# symm_gt_partial = dict(random.sample(sorted(symm_gt.items()), 20))

# for k, v in symm_gt_partial.items():
#     commands.append(gen_command(k, v, threshold=threshold_obja_lvis))

# with open("commands.sh", "w") as f:
#     for command in commands:
#         f.write(command + "\n")